In [ ]:
#Shift-tab to see parameters

%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sdgr import *

PATH = 'data/planet/'

ls {PATH}

#Labels: weather, agriculture, primary rainforest, water, etc.

list_paths = {f"{PATH}train-jpg/train_0.jpg", f"{PATH}train-jpg/train_1.jpg"}
titles = ["haze primary", "agriculture clear primary water"]
plots_from_files(list_paths, titles = titles, maintitle = "Sample of this multi label classification dateset")

from planet import f2 #a kaggle library

metrics = [f2]
f_model = resnet34

label_csv = f'{PATH}train_v2.csv'
n = len(list(open(label_csv)))-1
val_idxs = get_cv_idxs(n)

def get_data(sz):
	tfms = tfms_from_model(f_model, sz, aug_tfms = transforms_top_down, max_zoom = 1.05)
	return ImageClassifierData.from_csv(PATH, 'train-jpg', label_csv, tfms = tfms, suffix = '.jpg', val_idxs = val_idxs, test_name = 'test-jpg")
#Pass in sz = sz so the image is resized

data = get_data(256)

x,y = next(iter(data.val_dl)) #This is a iterator for the dataloader which gives minibatches of data. x is the images, y is the labels in the minibatch

list(zip(data.classes, y[0])) #zip combines lists by making a dictionary. This will now return all the labels in the 0th image. All the probabilities for possible labels are one hot encoded: it changes softmax values to 0 or 1

plt.imshow(data.val_ds.denorm(to_np(x))[0]*1.4);
#We use the denorm whatnot because the image was originally washed out so we made it a np matrix for the image and increased the values by 1.4 to brighten the image

#Now for the actual training
sz = 64
data = get_data(sz)
data = data.resize(int(sz*1.3), 'tmp') #This says that if an image is too big, it will just replace the big images with the resized images. This is just a speedup function
learn = ConvLearner.pretrained(f_model, data, metrics = metrics)

lrf = learn.lr_find()
learn.sched.plot()
#The minimum of the lr function graph is at a lr of 1 so we go down a bit

learn.fit(lr, 3, cycle_len = 1, cycle_mult = 2)
lrs = np.array([lr/9, lr/3, lr])
learn.unfreeze()
learn.fit(lrs, 3, cycle_len = 1, cycle_mult = 2) #Trig learning curves

learn.save(f'{sz}') #This makes the file inline
learn.sched.plot.loss()

sz = 128
learn.set_data(get_data(sz)
learn.freeze()
learn.fit(lr, 3, cycle_len = 1, cycle_mult = 2)

learn.unfreeze()
learn.fit(lrs, 3, cycle_len = 1, cycle_mult = 2) #Trig learning curves
learn.save(f'{sz}')

sz = 256 
learn.set_data(get_data(sz)
learn.freeze()
learn.fit(lr, 3, cycle_len = 1, cycle_mult = 2)

learn.unfreeze()
learn.fit(lrs, 3, cycle_len = 1, cycle_mult = 2) #Trig learning curves
learn.save(f'{sz}')

tta = learn.TTA()
f2(*tta)